<a href="https://colab.research.google.com/github/JorgeAnsotegui/TFM/blob/main/Yolo2Coco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Conectar Colab a Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import random
import shutil
import json
from PIL import Image
import cv2
from datetime import datetime

# Rutas de las carpetas de imágenes y etiquetas originales
images_dir = "/content/drive/MyDrive/TFM/dataset_Jorge_V2/train/images"
labels_dir = "/content/drive/MyDrive/TFM/dataset_Jorge_V2/train/labels"

# Ruta principal del dataset final
main_dir = "/content/dataset_MaskRCNN"

# Subcarpetas del dataset final
train_dir = os.path.join(main_dir, "train")
test_dir = os.path.join(main_dir, "test")
val_dir = os.path.join(main_dir, "val")

# Rutas de las carpetas de train, test y val para imágenes y etiquetas
train_images_dir = os.path.join(train_dir, "images")
train_labels_dir = os.path.join(train_dir, "labels")
test_images_dir = os.path.join(test_dir, "images")
test_labels_dir = os.path.join(test_dir, "labels")
val_images_dir = os.path.join(val_dir, "images")
val_labels_dir = os.path.join(val_dir, "labels")

# Función para crear carpetas si no existen
def create_directories_if_not_exist(*directories):
    for directory in directories:
        os.makedirs(directory, exist_ok=True)

# Crear carpetas de train, test y val si no existen
create_directories_if_not_exist(train_dir, test_dir, val_dir)
create_directories_if_not_exist(train_images_dir, train_labels_dir, test_images_dir, test_labels_dir, val_images_dir, val_labels_dir)

# Proporciones para train, test y val
train_percent = 0.6
test_percent = 0.2
val_percent = 0.2

# Obtener lista de nombres de archivos de imágenes
image_files = os.listdir(images_dir)
# Filtrar solo los archivos con extensión .jpg
image_files = [file for file in image_files if file.endswith(".jpg")]

# Barajar la lista de nombres de archivos
random.shuffle(image_files)

# Calcular el número de archivos para cada conjunto
total_images = len(image_files)
num_train = int(total_images * train_percent)
num_test = int(total_images * test_percent)
num_val = total_images - num_train - num_test

# Dividir la lista de nombres de archivos en train, test y val
train_images = image_files[:num_train]
test_images = image_files[num_train:num_train+num_test]
val_images = image_files[num_train+num_test:]

# Crear carpetas de train, test y val si no existen
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(test_images_dir, exist_ok=True)
os.makedirs(test_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Copiar imágenes y etiquetas a las carpetas correspondientes
def copy_files(files, source_dir, dest_images_dir, dest_labels_dir):
    for file in files:
        # Copiar imágenes
        shutil.copy(os.path.join(source_dir, file), dest_images_dir)
        # Copiar etiquetas
        label_file = os.path.splitext(file)[0] + ".txt"
        shutil.copy(os.path.join(labels_dir, label_file), dest_labels_dir)

copy_files(train_images, images_dir, train_images_dir, train_labels_dir)
copy_files(test_images, images_dir, test_images_dir, test_labels_dir)
copy_files(val_images, images_dir, val_images_dir, val_labels_dir)

print("División de imágenes y etiquetas completada.")


División de imágenes y etiquetas completada.


# Leer archivos de etiquetas de YOLO para train, test y val


In [3]:
def read_yolo_labels(yolo_label_dir):
    """Función para leer archivos .txt de etiquetas de YOLO."""
    yolo_labels = {}
    for file_name in os.listdir(yolo_label_dir):
        if file_name.endswith(".txt"):
            with open(os.path.join(yolo_label_dir, file_name), "r") as f:
                content = f.read().strip().split("\n")
                labels = []
                for line in content:
                    parts = line.split(" ")
                    class_id = int(parts[0])
                    points = []
                    for i in range(1, len(parts), 2):
                        x, y = map(float, parts[i:i+2])
                        point = [x, y]
                        points.append(point)
                    label = {"class_id": class_id, "points": points}
                    labels.append(label)
                yolo_labels[file_name] = labels
    return yolo_labels

train_yolo_labels = read_yolo_labels(train_labels_dir)
test_yolo_labels = read_yolo_labels(test_labels_dir)
val_yolo_labels = read_yolo_labels(val_labels_dir)

print("Etiquetas de YOLO leídas para train, test y val.")

Etiquetas de YOLO leídas para train, test y val.


# Función para convertir etiquetas YOLO a COCO Segmentation


In [4]:
def convert_to_coco_labels(yolo_labels, image_folder, start_image_id=1, start_bbox_id=1):
    current_year = datetime.now().year
    current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    coco_labels = {
        'info': {
            'description': 'Dataset',
            'url': '',
            'version': '1.0',
            'year': current_year,
            'contributor': '',
            'date_created': current_date
        },
        'licenses': [{'id': 1, 'name': None, 'url': None}],
        'images': [],
        'annotations': [],
        'categories': []
    }

    categories_set = set()
    image_id = start_image_id
    bbox_id = start_bbox_id

    for file_name, yolo_objs in yolo_labels.items():
        image_name = file_name.replace(".txt", ".jpg")
        image_path = os.path.join(image_folder, image_name)

        with Image.open(image_path) as img:
            width, height = img.size

        coco_labels['images'].append({
            'file_name': image_name,
            'height': height,
            'width': width,
            'id': image_id
        })

        for obj in yolo_objs:
            class_id = obj["class_id"]
            categories_set.add(class_id)

            x_yolo, y_yolo = obj["points"][0]
            w_yolo, h_yolo = obj["points"][1][0] - x_yolo, obj["points"][1][1] - y_yolo
            x_coco = round(x_yolo * width - (w_yolo * width / 2))
            y_coco = round(y_yolo * height - (h_yolo * height / 2))
            w_coco = round(w_yolo * width)
            h_coco = round(h_yolo * height)

            bbox = [x_coco, y_coco, w_coco, h_coco]
            area = w_coco * h_coco

            segmentation = [
                [x_coco, y_coco, x_coco + w_coco, y_coco, x_coco + w_coco, y_coco + h_coco, x_coco, y_coco + h_coco]
            ]

            coco_labels['annotations'].append({
                'id': bbox_id,
                'image_id': image_id,
                'category_id': class_id + 1,
                'bbox': bbox,
                'area': area,
                'segmentation': segmentation,
                'iscrowd': 0
            })

            bbox_id += 1

        image_id += 1

    for class_id in categories_set:
        coco_labels['categories'].append({
            'id': class_id + 1,
            'name': str(class_id),
            'supercategory': ''
        })

    return coco_labels

# Convertir etiquetas YOLO a etiquetas COCO para train, test y val
train_coco_labels = convert_to_coco_labels(train_yolo_labels, train_images_dir)
test_coco_labels = convert_to_coco_labels(test_yolo_labels, test_images_dir)
val_coco_labels = convert_to_coco_labels(val_yolo_labels, val_images_dir)

# Guardar las etiquetas COCO en archivos JSON únicos




In [6]:
# Función para guardar etiquetas COCO en un archivo JSON
def save_coco_labels(labels, output_file):
    output_dir = os.path.dirname(output_file)
    os.makedirs(output_dir, exist_ok=True)
    with open(output_file, "w") as f:
        json.dump(labels, f, indent=4)

# Guardar las etiquetas COCO en archivos JSON
save_coco_labels(train_coco_labels, "/content/dataset_COCO/train/coco_labels.json")
save_coco_labels(test_coco_labels, "/content/dataset_COCO/test/coco_labels.json")
save_coco_labels(val_coco_labels, "/content/dataset_COCO/val/coco_labels.json")

print("Conversión a etiquetas COCO Segmentation completada.")


Conversión a etiquetas COCO Segmentation completada.
